In [1]:
import pandas as pd
import numpy as np

cell_size = 0.0083333333333333
NODATA_value = -9999
start_x = -180
start_y = 90

filepath_base = "gpw-v4-population-count-rev11_2020_30_sec_asc/gpw_v4_population_count_rev11_2020_30_sec_"
grid = None
for col in range(2):
    grid_line = None
    for row in range(4):
        filepath = filepath_base + str(col * 4 + row + 1) + ".asc"
        # grid_part = np.loadtxt(filepath, skiprows=6)
        grid_part = pd.read_csv(filepath,header=None,na_filter=False,delim_whitespace=True, skiprows=6).to_numpy()
        if grid_line is None:
            grid_line = grid_part
        else:
            grid_line = np.concatenate((grid_line, grid_part), axis=1)
    if grid is None:
        grid = grid_line
    else :
        grid = np.concatenate((grid, grid_line), axis=0)

In [2]:
grid.shape

(21600, 43200)

In [8]:
print (np.where(grid != NODATA_value))

(array([  764,   764,   764, ..., 17518, 17518, 17518]), array([17677, 17678, 17679, ..., 13527, 13528, 13529]))


In [2]:
# !pip install shapely

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [4]:
from shapely.geometry import shape, Point
from shapely.prepared import prep
import numpy as np
import json
# cell_size = 0.0083333333333333
# NODATA_value = -9999
with open("1.geojson") as gj:
    gjson = json.load(gj)

polygon = shape(gjson['features'][0]['geometry'])
(latmin, lonmin, latmax, lonmax) =((x // cell_size * cell_size) for x in polygon.bounds)
prep_polygon = prep(polygon)
points = []
for lat in np.arange(latmin, latmax, cell_size):
    for lon in np.arange(lonmin, lonmax, cell_size):
        points.append(Point((round(lat,4), round(lon,4))))
valid_points = []
valid_points.extend(filter(prep_polygon.contains, points))
# print(valid_points)

In [6]:
sum = 0
for point in  valid_points:
    lon, lat = point.x, point.y
    local_sum = grid[int((start_y - lat) // cell_size)][int((lon - start_x) // cell_size)]
    if local_sum != NODATA_value:
        sum += int(local_sum)
print(sum)

28109088
